In [7]:
import pandas as pd, json

In [8]:
df_geo = pd.read_csv('geocoded.csv')
df_geo.city = df_geo.city.str.replace('Володимир Волинський', 'Володимир-Волинський'
                        ).str.replace('Білгород Дністровський', 'Білгород-Дністровський')

In [16]:
# df_geo.reindex(['city', 'lat', 'lon', 'district'], axis=1
#       ).drop_duplicates(
#       ).to_csv('geocoded.csv', index=False)

In [12]:
df_geo.district.value_counts()

no     78
yes    22
Name: district, dtype: int64

In [17]:
df = pd.read_excel('100 міст- файл для правильного внесення балів по містах (1).xlsx',
                   sheet_name='100 міст_2017_2018')
df = df.reindex(list(filter(lambda s: not ' VS ' in s, df.columns)), axis=1)
df = pd.melt(df, id_vars=df.columns[:4], value_vars=df.columns[4:])
df.columns = ['city', 'rank_2017', 'points_2017', 'points_2018', 'indicator', 'value']
df['year'] = df.indicator.str.extract('(\d{4}) ?$').astype(int)
df.city = df.city.str.strip(
                ).str.replace('\s*?-\s*', '-'
                ).str.replace('Кременчуг', 'Кременчук'
                ).str.replace('Подільск', 'Подільськ'
                ).str.replace('Севєродонецьк', 'Сєвєродонецьк'
                ).str.replace('Первомайск', 'Первомайськ')
df.indicator = df.indicator.str.replace('\s*\d{4} ?$', '')
df = df.drop(['rank_2017', 'points_2017', 'points_2018'], axis=1)

totals = df.groupby(['city', 'year'], as_index=True).sum().reset_index()
totals['indicator'] = 'Загальний бал'

df = df.append(totals, ignore_index=True, sort=False).sort_values(['city', 'indicator','year'])

df = df.merge(df_geo, how='left')

In [19]:
# df.to_csv('data_long.csv', index=False)

In [8]:
pd.Series(dict(pd.read_csv('indicators_max.csv').values)
         ).to_json('indicators_max.json', force_ascii=False)